<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split

# Define a search space
class SearchSpace(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SearchSpace, self).__init__()
        self.fc1 = nn.ModuleList([nn.Linear(input_dim, 128), nn.Linear(input_dim, 256)])
        self.fc2 = nn.ModuleList([nn.Linear(128, output_dim), nn.Linear(256, output_dim)])

    def forward(self, x, arch):
        x = torch.relu(self.fc1[arch[0]](x))
        if arch[0] == 0:
            x = self.fc2[0](x)  # Use the first layer in fc2
        else:
            x = self.fc2[1](x)  # Use the second layer in fc2
        return x

# Hyperparameters
input_dim = 784  # 28x28 MNIST images
output_dim = 10
batch_size = 64
lr = 0.01
num_epochs = 10

# Data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Search for the best architecture
best_accuracy = 0
best_arch = None
search_space = SearchSpace(input_dim, output_dim)
criterion = nn.CrossEntropyLoss()

for arch in [[0, 0], [0, 1], [1, 0], [1, 1]]:
    # Reinitialize the model parameters for each architecture
    search_space.apply(lambda m: m.reset_parameters() if hasattr(m, 'reset_parameters') else None)
    optimizer = optim.Adam(search_space.parameters(), lr=lr)

    for epoch in range(num_epochs):
        search_space.train()
        for images, labels in train_loader:
            images = images.view(-1, input_dim)
            outputs = search_space(images, arch)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation
        search_space.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.view(-1, input_dim)
                outputs = search_space(images, arch)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Arch: {arch}, Epoch: {epoch+1}, Accuracy: {accuracy:.2f}%")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_arch = arch

print(f"Best Architecture: {best_arch}, Best Accuracy: {best_accuracy:.2f}%")